# Memoria **COVID_&_mobility**

## 1_INTRODUCCIÓN

Cruce de datos de incidencia de la covid_19 en España proporcionados por el Ministerio de Sanidad con datos de movilidad sobre la afluencia de personas a determinadas lugares agrupados en diferentes categorías proporcionados por Google. <br>
Todo ello agrupado por fechas y provincias. Fechas entre 15-02-2020 y 27-03-2022.

Veremos la correspondencia entre la incidencia del covid y las métricas de afluencia de público a según qué sitios.


### HIPÓTESIS

##### La principal hipótesis es que la incidencia del Covid influyó sobre los lugares que mas se frecuentaron durante la pandemia.

De ahí salen varias preguntas: <br>
- ¿Existen cambios significativos en los patrones de conducta durante este periodo? <br>
- ¿Existe correspondencia entre las diferentes métricas temporales? <br>
- ¿Podemos con estos datos afirmar que existe causalidad o mera correspondencia entre ellos? <br>
- ¿Qué efectos tiene para cada categoría de estudio? <br>
- ¿Cuánto duran los efectos de esos cambios? <br>


##### Por otro lado, podríamos verlo al revés, ¿influye los lugares frecuentados con el crecimiento de los datos de incidencia del covid?



## 2_OBTENCIÓN DE LOS DATOS

### DATASETS

*df_google* es un dataset publicado por Google que se puede descargar del siguiente enlace: [www.google.com/covid19/mobility/data_documentation](https://www.google.com/covid19/mobility/data_documentation.html?hl=es) <br>
*df_covd19* es un dataset publicado por el [Instituto de Salud Carlos III (ISCIII)](https://www.isciii.es) y que puede encontrar en el siguiente enlace: [casos_hosp_uci_def_sexo_edad_provres.csv](https://cnecovid.isciii.es/covid19/resources/casos_hosp_uci_def_sexo_edad_provres.csv)


In [2]:
import pandas as pd

df_20 = pd.read_csv('2020_ES_Region_Mobility_Report.csv')
df_21 = pd.read_csv('2021_ES_Region_Mobility_Report.csv')
df_22 = pd.read_csv('2022_ES_Region_Mobility_Report.csv')
df_google = pd.concat([df_20, df_21, df_22], ignore_index= True)

df_covid19 = pd.read_csv('casos_hosp_uci_def_sexo_edad_provres.csv')

In [3]:
df_google.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,ES,Spain,NaN,NaN,NaN,NaN,NaN,ChIJi7xhMnjjQgwR7KNoB5Qs7KY,2020-02-15,2.0,-1.0,26.0,8.0,0.0,-2.0
1,ES,Spain,NaN,NaN,NaN,NaN,NaN,ChIJi7xhMnjjQgwR7KNoB5Qs7KY,2020-02-16,2.0,3.0,13.0,5.0,-1.0,-2.0
2,ES,Spain,NaN,NaN,NaN,NaN,NaN,ChIJi7xhMnjjQgwR7KNoB5Qs7KY,2020-02-17,0.0,1.0,9.0,3.0,3.0,-1.0
3,ES,Spain,NaN,NaN,NaN,NaN,NaN,ChIJi7xhMnjjQgwR7KNoB5Qs7KY,2020-02-18,-2.0,0.0,5.0,4.0,3.0,0.0
4,ES,Spain,NaN,NaN,NaN,NaN,NaN,ChIJi7xhMnjjQgwR7KNoB5Qs7KY,2020-02-19,0.0,1.0,11.0,2.0,3.0,-1.0


In [4]:
df_covid19.head()

,provincia_iso,sexo,grupo_edad,fecha,num_casos,num_hosp,num_uci,num_def
0,A,H,0-9,2020-01-01,0,0,0,0
1,A,H,10-19,2020-01-01,0,0,0,0
2,A,H,20-29,2020-01-01,0,0,0,0
3,A,H,30-39,2020-01-01,0,0,0,0
4,A,H,40-49,2020-01-01,0,0,0,0


-  Tras cargar *df_google* prescindimos de las columnas *metro_area y census_fips_code* ya que no aportan información. <br> 
    Posteriormente rellenamos la columna *sub_region_1* por faltar nombres de las provincias de las comunidades con una sola provincia. Comprobamos que están todas las provincias. <br>
    Creamos una nueva columna a partir de *sub_region_1*, *provincia_id_g* para cruzarla con la otra tabla.

- En *df_covid19* encontramos que hay datos que no están asociados a provincias concretas, prescindimos de ellos ya que queremos solo los datos asociados a una provincia.

Una vez que tenemos los dos dataset preparados que vamos a usar en este EDA toca unirlos, para ello se hace un *merge* de df_google sobre df_covid19 por las columnas fecha y provincia. Columnas estas previamente armonizadas.  

## 3_LIMPIEZA DE LOS DATOS

- Borramos la columna *Unnamed: 0*.
- Recortamos el dataframe por las primeras fechas ya que no hay datos de google antes del 15-02-2020. 
- Tratamos varios nulos en las columnas de las provincias que tiene como resultado prescindir de los datos sobre Ceuta y Melilla. 
- Además observamos una serie de nulos que corresponden a situaciónes en las que google ha considerado que no tenían sufiente calidad los datos que había recogido. <br>
    Son casos puntuales, por lo que los rellenamos con la media de los valores del día anterior y posterior.
- Dropeamos entonces las columnas *['date', 'provincia_iso_g', 'country_region_code', 'iso_3166_2_code']*.
- Renombramos y reordenamos las columnas.

Y ya tenemos el dataset limpio y listo para el estudio.

## 4_ESTUDIO

- Cambio de las fechas a *type datetime*.

Comienzo del Análisis univariante:
- Contrucción de la tabla resumen de análisis de los datos de las diferentes columnas con *nombre, type, prio, card, card%, class y mode*.
- Gráficas sobre la distribución de los datos. Primero las Categóricas y luego las numéricas.

Creamos nuevos subdataframes, agrupando por *fechas* y por *fechas y provincias*. Trabajaremos principalmente con la agrupación por *fechas*. 
- Primero volvemos hacer el análizar el análisis univariante de las columnas numéricas.

Aquí comienza el análisis bivariante:
- Lo primero es hacer un análisis de los datos de la incidencia del covid en función del tiempo. Además suavizamos la curva para asumir fallas semanales en la recogida de datos. Ventana de 7 días. Usamos gráfica de líneas.
- Realizamos diferentes gráficas para comprar las diferentes métricas de incidencia. Usaremos *num_casos* y *num_hosp* para lo siguiente.
- Seguimos con un análsis temporal de los datos de afluencia de gente. También suavizamos con ventana de 7 días. Le superponemos la línea de *num_casos* para ver posibles correlaciones. Usamos gráfica de líneas.
- Comparamos ahora directamente mediante mapas de calor cada una de las categorías de lugares de Google, primero con *num_casos*, y luego con también con el resto de métricas de incidencia. 
- Así mismo hacemos una comparación de los mapas de calor anteriores con otro con un desfase de 15 días atendiendo al periodo de desactivación del contagio del covid.
- Se observa, 
    - primero que podría existir cierta correspondencia entre las métricas, 
    - segundo se duda de la fiabildad de *num_casos*, 
    - tercero parece haber más corresondencia con el desfase de 15 días.
- Para tratar de aclarar esto buscamos una representación algo más clarificadora, y hacemos lo mismo con diagramas de puntos. Y nos aclaran dos cosas: <br>
    - *num_casos* no es un buen dato ya que no parece reflejar bien la intensidad real de la epidemia por la imposibilidad de registrar todos los casos. Optamos por trabajar con *num_hosp*. <br>
    - se confirma la correspondencia con el desfase.
- Volvemos a hacer la comparación temporal de cada categoría de lugar esta vez con *num_hosp* y el análisis visual muestra mayor correspondencia.
- Para terminar comprobamos la correlación de Pearson entre las columnas que no sale nada llamativo. Pero al hacer la correlación cruzada observamos correlaciones de en torno a 0.7 en la mayoría de ellas a 10 días. 

## 5_CONCLUSIONES

- El mejor indicador epidemiológico parece ser el número de hospitalizaciones.
- No podemos afirmar a ciencia cierta que haya causalidad, pero sí podemos hablar de correspondencia. 
- En ese sentido podríamos concluir que en general hay una correspondencia moderada-fuerte entre la incidencia del covid y la afluencia de público a lugares de comercio_ocio, estaciones y lugares de trabajo inversamente proporcional y una relación directa con las zonas residenciales.
- No pasa así con los supermercados_farmacias y parques cuya afluencia de público parece ser independiente del número de hospitalizaciones.
- En los primeros, la tendencia es a la normalización con la baseline. En los segundos tienen una tendencia alcista. Si bien, en las zonas de trabajo la afluencia de público parece constante y menor a la baseline sin vistas de movimiento. Algo parecido pasa con las zonas de comercio_ocio que aún al final no han recuperado su afluencia.
- En general este comportamiento se corresponde con lo anteriormente citado, si bien se ve relajado con el tiempo.
- Esta correlación moderada-fuerte se da con 10 días de antelación. Es decir, la correspondencia se da entre los datos de afluencia de 10 días anteriores con respecto al número de hospitalizados.


## 6_ESTUDIOS FUTUROS

- Hacer estudios por provincia para ver si todas siguen el mismo patrón de comportamiento.
- Cruzar los datos con las poblaciones de cada provincia y estudiar los datos proporcionalmente.
- Hacer estudios teniendo en cuenta el sexo y los grupos de edad.
- Incluir hitos como medidas de restricciones de movilidad generales y por ámbitos geográficos. Así como uso de mascarilla en según qué sitios.
- Dividir la muestra en periodos. Estudiar la estacionalidad.
- Entrenar un modelo que nos permita rellenar los NAN que encontramos en Ceuta y Melilla